# **Heston PINN**
---


$$
\frac{\partial V}{\partial t} + rS \frac{\partial V}{\partial S} + \rho\sigma v S \frac{\partial^2 V}{\partial S \partial v} + \frac{1}{2}S^2 v \frac{\partial^2 V}{\partial S^2} + \frac{1}{2}\sigma^2 v \frac{\partial^2 V}{\partial v^2} + \kappa(\theta - v) \frac{\partial V}{\partial v} - rV = 0
$$
<center>

**Heston PDE**
</center>

Where:
* $V$: Option price (a function of $S$, $v$, and $t$)
* $t$: Time
* $S$: Price of the underlying asset
* $v$: Instantaneous variance of the underlying asset (volatility squared)
* $r$: Risk-free interest rate
* $\rho$: Correlation between the Brownian motion of the asset price and its variance
* $\sigma$: Volatility of the variance (volatility of volatility)
* $\kappa$: Rate at which the variance $v$ reverts to its long-term mean $\theta$
* $\theta$: Long-term mean variance

---

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import time
import wandb

wandb.login(key='601e2bae7faf9f70cd48f1c1ae9ed183b5193d1c')

# =============================================================================
# 1. PARAMETERS (GLOBAL)
# These are fixed for all sweep runs.
# =============================================================================
r = 0.05
sigma = 0.2
K = 100.0
T = 1.0
theta = sigma**2
v_0 = sigma**2
kappa = 2.0
sigma_v = 0.3
rho = -0.7
S_min, S_max = 0.0, 250.0
v_min, v_max = 0.0, 1.0
t_min, t_max = 0.0, T

# =============================================================================
# 2. NEURAL NETWORK MODEL
# =============================================================================
class HestonPINN(nn.Module):
    def __init__(self, input_dim=3, output_dim=1, hidden_layers=4, neurons_per_layer=64, activation_fn=nn.Tanh()):
        super().__init__()
        layers = [nn.Linear(input_dim, neurons_per_layer), activation_fn]
        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(activation_fn)
        layers.append(nn.Linear(neurons_per_layer, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, S, v, t):
        x = torch.cat([S, v, t], dim=1)
        return self.net(x)

# =============================================================================
# 3. LOSS FUNCTIONS
# =============================================================================
def pde_loss(model, S, v, t):
    C = model(S, v, t)
    C_t = torch.autograd.grad(C, t, grad_outputs=torch.ones_like(C), create_graph=True)[0]
    C_S = torch.autograd.grad(C, S, grad_outputs=torch.ones_like(C), create_graph=True)[0]
    C_v = torch.autograd.grad(C, v, grad_outputs=torch.ones_like(C), create_graph=True)[0]
    C_SS = torch.autograd.grad(C_S, S, grad_outputs=torch.ones_like(C_S), create_graph=True)[0]
    C_vv = torch.autograd.grad(C_v, v, grad_outputs=torch.ones_like(C_v), create_graph=True)[0]
    C_Sv = torch.autograd.grad(C_S, v, grad_outputs=torch.ones_like(C_S), create_graph=True)[0]
    pde_residual = (C_t + r * S * C_S + kappa * (theta - v) * C_v + 0.5 * v * S**2 * C_SS + 0.5 * sigma_v**2 * v * C_vv + rho * sigma_v * v * S * C_Sv - r * C)
    return torch.mean(pde_residual**2)

def boundary_loss(model, v_bc, t_bc):
    S_zero = torch.zeros_like(t_bc).requires_grad_()
    C_at_S_zero = model(S_zero, v_bc, t_bc)
    loss_S_zero = torch.mean(C_at_S_zero**2)
    S_at_max = (torch.ones_like(t_bc) * S_max).requires_grad_()
    C_at_S_max_pred = model(S_at_max, v_bc, t_bc)
    C_at_S_max_true = S_at_max - K * torch.exp(-r * (T - t_bc))
    loss_S_max = torch.mean((C_at_S_max_pred - C_at_S_max_true)**2)
    return loss_S_zero + loss_S_max

def terminal_loss(model, S_tc, v_tc):
    t_terminal = (torch.ones_like(S_tc) * T).requires_grad_()
    C_pred = model(S_tc, v_tc, t_terminal)
    C_true = torch.clamp(S_tc - K, min=0)
    return torch.mean((C_pred - C_true)**2)

# =============================================================================
# 4. SWEEP TRAINING FUNCTION
# =============================================================================
def train_pinn_sweep():
    wandb.init()
    config = wandb.config

    model = HestonPINN(
        hidden_layers=config.hidden_layers,
        neurons_per_layer=config.neurons_per_layer
    )
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    # Set T_max to the total number of epochs for one full cycle.
    scheduler = CosineAnnealingLR(optimizer, T_max=config.num_epochs, eta_min=1e-7)


    for epoch in range(config.num_epochs):
        optimizer.zero_grad()
        
        S_pde = (torch.rand(config.N_pde, 1) * S_max).requires_grad_()
        v_pde = (torch.rand(config.N_pde, 1) * v_max).requires_grad_()
        t_pde = (torch.rand(config.N_pde, 1) * T).requires_grad_()
        loss_pde_val = pde_loss(model, S_pde, v_pde, t_pde)

        v_bc = (torch.rand(config.N_boundary, 1) * v_max).requires_grad_()
        t_bc = (torch.rand(config.N_boundary, 1) * T).requires_grad_()
        loss_bc_val = boundary_loss(model, v_bc, t_bc)

        S_tc = (torch.rand(config.N_terminal, 1) * S_max).requires_grad_()
        v_tc = (torch.rand(config.N_terminal, 1) * v_max).requires_grad_()
        loss_tc_val = terminal_loss(model, S_tc, v_tc)
        
        total_loss = (config.w_pde * loss_pde_val) + (config.w_boundary * loss_bc_val) + (config.w_terminal * loss_tc_val)
        
        total_loss.backward()
        optimizer.step()
        scheduler.step()
        
        wandb.log({
            "epoch": epoch,
            "total_loss": total_loss.item(),
            "pde_loss": loss_pde_val.item(),
            "boundary_loss": loss_bc_val.item(),
            "terminal_loss": loss_tc_val.item(),
            "learning_rate": optimizer.param_groups[0]['lr']
        })

    wandb.finish()

# =============================================================================
# 5. SWEEP CONFIGURATION
# =============================================================================
def generate_sweep_config(flags, defaults):
    sweep_config = {
        'method': flags['sweep_method'],
        'metric': {'name': 'terminal_loss', 'goal': 'minimize'},
        'parameters': {}
    }
    
    params = sweep_config['parameters']
    
    # --- Hyperparameters to sweep ---
    if flags['sweep_hidden_layers']:
        params['hidden_layers'] = {'distribution': 'q_uniform', 'min': 2, 'max': 8, 'q': 2}
    else:
        params['hidden_layers'] = {'value': defaults['hidden_layers']}

    if flags['sweep_neurons_per_layer']:
        # CORRECTED: Use 'values' for categorical choices in random/bayes sweeps
        params['neurons_per_layer'] = {'values': [32, 64, 128, 256]}
    else:
        params['neurons_per_layer'] = {'value': defaults['neurons_per_layer']}
    
    if flags['sweep_learning_rate']:
        params['learning_rate'] = {'distribution': 'log_uniform_values', 'min': 1e-5, 'max': 1e-2}
    else:
        params['learning_rate'] = {'value': defaults['learning_rate']}
        
    # --- Loss weights to sweep ---
    if flags['sweep_w_pde']:
        params['w_pde'] = {'distribution': 'uniform', 'min': 1.0, 'max': 10.0}
    else:
        params['w_pde'] = {'value': defaults['w_pde']}

    if flags['sweep_w_boundary']:
        params['w_boundary'] = {'distribution': 'uniform', 'min': 0.001, 'max': 2.0}
    else:
        params['w_boundary'] = {'value': defaults['w_boundary']}

    if flags['sweep_w_terminal']:
        params['w_terminal'] = {'distribution': 'uniform', 'min': 0.001, 'max': 2.0}
    else:
        params['w_terminal'] = {'value': defaults['w_terminal']}

    # --- Fixed parameters for all runs ---
    params['num_epochs'] = {'value': defaults['num_epochs']}
    params['N_pde'] = {'value': defaults['N_pde']}
    params['N_boundary'] = {'value': defaults['N_boundary']}
    params['N_terminal'] = {'value': defaults['N_terminal']}
    
    return sweep_config

# =============================================================================
# 6. MAIN EXECUTION BLOCK
# =============================================================================
if __name__ == '__main__':
    # --- Login to W&B ---
    # Make sure to set your API key
    # wandb.login(key='YOUR_API_KEY')

    # --- Flags to toggle which hyperparameters to sweep ---
    config_flags = {
        'sweep_hidden_layers': False,
        'sweep_neurons_per_layer': False,
        'sweep_learning_rate': False,
        'sweep_w_pde': True,
        'sweep_w_boundary': True,
        'sweep_w_terminal': True,
        'sweep_method': 'bayes'  # 'random', 'bayes', or 'grid'
    }

    # --- Default values if a parameter is not being swept ---
    fixed_defaults = {
        'hidden_layers': 2,
        'neurons_per_layer': 256,
        'learning_rate': 0.0024,
        'w_pde': 10.0,
        'w_boundary': 0.9,
        'w_terminal': 0.04,
        'num_epochs': 5000, # Reduced for quicker sweep runs
        'N_pde': 2500,
        'N_boundary': 500,
        'N_terminal': 500
    }
    
    num_sweeps = 25 # How many runs you want the agent to execute

    # --- Generate config and run sweep ---
    sweep_config = generate_sweep_config(config_flags, fixed_defaults)
    sweep_id = wandb.sweep(sweep_config, project="Heston-Weight-Sweeps-3")
    wandb.agent(sweep_id, function=train_pinn_sweep, count=num_sweeps)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\tobys\_netrc
wandb: Currently logged in as: tobiassafie (tobiassafie-drexel-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Create sweep with ID: ljlwhom6
Sweep URL: https://wandb.ai/tobiassafie-drexel-university/Heston-Weight-Sweeps-3/sweeps/ljlwhom6


wandb: Agent Starting Run: muzpjojv with config:
wandb: 	N_boundary: 500
wandb: 	N_pde: 2500
wandb: 	N_terminal: 500
wandb: 	hidden_layers: 2
wandb: 	learning_rate: 0.0024
wandb: 	neurons_per_layer: 256
wandb: 	num_epochs: 5000
wandb: 	w_boundary: 0.09222819063533796
wandb: 	w_pde: 9.188053761149046
wandb: 	w_terminal: 0.05254829359331053


boundary_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▆▆▆▆▇▇▇▇█████
learning_rate,██████████▇▇▇▇▇▆▆▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
pde_loss,▂▂▁▁▁▂▂▂▁▁▁▁▂▁▂▂▁▁▂█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
terminal_loss,▆▆▄▃▃▄█▃▃▃▂▂▁▂▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_loss,█▆▄▃▃▃▂▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
boundary_loss,2.7011
epoch,4999
learning_rate,0.0
pde_loss,0.06664
terminal_loss,26.99649


wandb: Agent Starting Run: 4krcqptb with config:
wandb: 	N_boundary: 500
wandb: 	N_pde: 2500
wandb: 	N_terminal: 500
wandb: 	hidden_layers: 2
wandb: 	learning_rate: 0.0024
wandb: 	neurons_per_layer: 256
wandb: 	num_epochs: 5000
wandb: 	w_boundary: 0.7664476786750976
wandb: 	w_pde: 4.40085015526702
wandb: 	w_terminal: 1.977122002987308


...

wandb: Agent Starting Run: d0gpvopi with config:
wandb: 	N_boundary: 500
wandb: 	N_pde: 2500
wandb: 	N_terminal: 500
wandb: 	hidden_layers: 2
wandb: 	learning_rate: 0.0024
wandb: 	neurons_per_layer: 256
wandb: 	num_epochs: 5000
wandb: 	w_boundary: 0.12188438740932256
wandb: 	w_pde: 6.350144404752142
wandb: 	w_terminal: 1.9726061137019917


wandb: Network error (ConnectionError), entering retry loop.


boundary_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
learning_rate,█████▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
pde_loss,█▄▄▃▆▅▂▂▃▁▂▃▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
terminal_loss,█▇▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
boundary_loss,1.37711
epoch,4999
learning_rate,0.0
pde_loss,0.22054
terminal_loss,1.77631


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 0ft9uyvz with config:
wandb: 	N_boundary: 500
wandb: 	N_pde: 2500
wandb: 	N_terminal: 500
wandb: 	hidden_layers: 2
wandb: 	learning_rate: 0.0024
wandb: 	neurons_per_layer: 256
wandb: 	num_epochs: 5000
wandb: 	w_boundary: 1.6478612778487238
wandb: 	w_pde: 2.825227630988385
wandb: 	w_terminal: 1.8985511967026192


boundary_loss,█▆▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇███
learning_rate,████████▇▇▇▇▇▇▇▆▆▅▅▅▄▄▄▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
pde_loss,▁▃▂▂▂▂▂▂█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
terminal_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_loss,█▆▆▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
boundary_loss,0.09973
epoch,4999
learning_rate,0.0
pde_loss,0.60999
terminal_loss,1.73011
